In [1]:
import os
import tensorflow as tf
from elmo import Model
from collections import defaultdict
from elmo_utils import *


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # 忽略警告
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
try:
    from tensorflow.python.util import module_wrapper as deprecation
except ImportError:
    from tensorflow.python.util import deprecation_wrapper as deprecation
deprecation._PER_MODULE_WARNING_LIMIT = 0

In [2]:
checkpoint_dir = "output/elmo_origin_loss_fix_at_best"

In [3]:
inference_graph = tf.Graph()

with inference_graph.as_default():
    inference_config = load_config(checkpoint_dir,
                                {"mode":'inference', 'checkpoint_dir':checkpoint_dir+"/best_eval"})
    inference_model = Model(inference_config)
    inference_model.build()

loading config from output/elmo_origin_loss_fix_at_best\config
# Start to load pretrained embedding...
# vocab size:  50000
# pretrained embedding size 44512 300
构建elmo静态图...


In [4]:
export_path = os.path.join(
        tf.compat.as_bytes('serving/'),
        tf.compat.as_bytes('v1/001')
)

In [5]:
with tf.Session(graph=inference_graph, config=get_config_proto(log_device_placement=False)) as inference_sess:
    inference_model.init_model(inference_sess)
    inference_model.restore_model(inference_sess)
    
    builder = tf.saved_model.builder.SavedModelBuilder(export_path)
    
    inputs_text_ids = inference_model.inputs
    inputs_text_lens = inference_model.seq_len
    
    predicts = inference_model.all_predicts
    logits = inference_model.all_logits
    
    inputs = {'text_ids': tf.saved_model.utils.build_tensor_info(inputs_text_ids),
                   'text_lens': tf.saved_model.utils.build_tensor_info(inputs_text_lens)}
    outputs = {'predicts': tf.saved_model.utils.build_tensor_info(predicts),
                     'logits': tf.saved_model.utils.build_tensor_info(logits)}
    
    signature = tf.saved_model.signature_def_utils.build_signature_def(
        inputs=inputs,
        outputs=outputs,
        method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
    
    builder.add_meta_graph_and_variables(inference_sess, 
                                         [tf.saved_model.tag_constants.SERVING], 
                                         {'predict_labels':signature})
    builder.save()


!!! Restored model


In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
from dataset import DataSet


eval_dataset = DataSet(["data/validation.json"],
                           "data/vocab.txt",
                           "data/labels.txt",
                           128,
                           reverse=False,
                           split_word=True,
                           max_len=1200)

# vocab size:  50000
# vocab size:  20
# Start to preprocessing data...
# load data from data/validation.json ...
# Got 15000 data items with 117 batches


In [10]:
for i in eval_dataset.get_next():
    print(i[0])
    print(i[1])
    print(i[2])
    print(i[3])

    break

[[    1    48     7 ...   908   586     2]
 [    1  1490   172 ...    31 16300     2]
 [    1    21   350 ...   325  3904     2]
 ...
 [    1   269   108 ...     2     2     2]
 [    1  6520    63 ...     2     2     2]
 [    1  3847 26296 ...     2     2     2]]
[1202 1202 1202 1202 1202 1202 1189 1166 1154 1126 1100 1096 1086 1054
 1038 1026 1023 1016 1010 1008  979  978  972  966  958  946  926  921
  910  910  888  877  876  870  856  850  839  837  835  833  827  822
  821  820  817  815  811  810  807  806  804  789  787  781  778  773
  756  755  754  753  741  738  727  726  723  712  711  711  708  707
  707  706  704  700  700  694  693  689  687  683  680  677  672  672
  671  669  668  668  666  657  654  651  650  650  647  647  646  645
  644  642  641  640  637  637  636  630  630  626  623  622  622  622
  622  618  613  611  611  609  608  606  606  601  598  596  596  595
  593  592]
[[[0. 0. 0. 1.]
  [0. 0. 0. 1.]
  [0. 0. 0. 1.]
  ...
  [0. 0. 0. 1.]
  [0. 0. 1. 0.]